| [02_advanced/04_迭代器与生成器.ipynb](https://github.com/shibing624/python-tutorial/blob/master/02_advanced/04_迭代器与生成器.ipynb)  | 迭代器和yield生成器  |[Open In Colab](https://colab.research.google.com/github/shibing624/python-tutorial/blob/master/02_advanced/04_迭代器与生成器.ipynb) |

# 迭代器

迭代是Python最强大的功能之一，是访问集合元素的一种方式。

迭代器是一个可以记住遍历的位置的对象。

迭代器对象从集合的第一个元素开始访问，直到所有的元素被访问完结束。迭代器只能往前不会后退。

迭代器有两个基本的方法：iter() 和 next()。

字符串，列表或元组对象都可用于创建迭代器：


In [62]:
list=[1,2,3,4]
it = iter(list)    # 创建迭代器对象

In [63]:
next(it) # 输出迭代器的下一个元素

1

In [64]:
next(it) # 再输出下一个元素

2

## enumerate
列表好处是不需要对下标进行迭代，直接输出列表的值：

In [65]:
x = [2, 4, 6]

for i in x:
    print(i)

2
4
6


但是有些情况下，我们既希望获得下标，
也希望获得对应的值，那么：

可以将迭代器传给 enumerate 函数，
这样每次迭代都会返回一组 (index, value) 组成的元组：


In [66]:
x = [2, 4, 6]
for i, n in enumerate(x):
    print(i, 'is', n)

0 is 2
1 is 4
2 is 6


## 自定义迭代器

一个迭代器都有  `__iter__()` 与 `__next__()` 

`__iter__()` 方法返回一个特殊的迭代器对象， 这个迭代器对象实现了 `__next__()` 方法并通过 StopIteration 异常标识迭代的完成。

`__next__()` 方法（Python 2 里是 next()）会返回下一个迭代器对象。

自定义一个 list 的取反迭代器：

In [67]:
class ReverseListIterator(object):
    def __init__(self, lst):
        self.list = lst
        self.index = len(lst)

    def __iter__(self):
        return self

    def __next__(self):
        self.index -= 1
        if self.index >= 0:
            return self.list[self.index]
        else:
            raise StopIteration

In [68]:
x = range(10)
for i in ReverseListIterator(x):
    print(i)

9
8
7
6
5
4
3
2
1
0


只要我们定义了这三个方法(`__init__, __iter__, __next__`)，我们可以返回任意迭代值：


## 实现Collatz 猜想
这里我们实现 Collatz 猜想：

- 奇数 n：返回 3n + 1
- 偶数 n：返回 n / 2
- 直到 n 为 1 为止：

In [69]:
class Collatz(object):
    def __init__(self, start):
        self.value = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.value == 1:
            raise StopIteration
        elif self.value % 2 == 0:
            self.value = self.value / 2
        else:
            self.value = 3 * self.value + 1
        return self.value


for x in Collatz(5):
    print(x)


16
8.0
4.0
2.0
1.0


不过迭代器对象存在状态，**有问题**：

In [70]:
i = Collatz(7)
for x, y in zip(i, i):
    print(x, y)

22 11.0
34.0 17.0
52.0 26.0
13.0 40.0
20.0 10.0
5.0 16.0
8.0 4.0
2.0 1.0


解决方法是将迭代器和可迭代对象分开处理。

## 迭代器和可迭代对象分开处理

这里提供了一个二分树的中序遍历实现：

In [71]:
class BinaryTree(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __iter__(self):
        return InorderIterator(self)

class InorderIterator(object):
    def __init__(self, node):
        self.node = node
        self.stack = []

    def __next__(self):
        if len(self.stack) > 0 or self.node is not None:
            while self.node is not None:
                self.stack.append(self.node)
                self.node = self.node.left
            node = self.stack.pop()
            self.node = node.right
            return node.value
        else:
            raise StopIteration()

测试：

In [72]:
tree = BinaryTree(
    left=BinaryTree(
        left=BinaryTree(1),
        value=2,
        right=BinaryTree(
            left=BinaryTree(3),
            value=4,
            right=BinaryTree(5)
        ),
    ),
    value=6,
    right=BinaryTree(
        value=7,
        right=BinaryTree(8)
    )
)

In [73]:
for value in tree:
    print(value)

1
2
3
4
5
6
7
8


不会出现之前的问题：

In [74]:
for x, y in zip(tree, tree):
    print(x, y)

1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8


# 生成器

在 Python 中，使用了 yield 的函数被称为生成器（generator）。

跟普通函数不同的是，生成器是一个返回迭代器的函数，只能用于迭代操作，更简单点理解生成器就是一个迭代器。

1. 迭代器则通过 next 的 return 将值返回；
2. 与迭代器不同的是，生成器会自动记录当前的状态，
而迭代器则需要进行额外的操作来记录当前的状态。

之前的 collatz 猜想，简单循环的实现如下：

collatz:

- 奇数 n：返回 3n + 1
- 偶数 n：返回 n / 2
- 直到 n 为 1 为止：

In [75]:
def collatz(n):
    sequence = []
    while n != 1:
        if n % 2 == 0:
            n /= 2
        else:
            n = 3 * n + 1
        sequence.append(n)
    return sequence


for x in collatz(5):
    print(x)

16
8.0
4.0
2.0
1.0


生成器的版本如下：

In [76]:
def collatz(n):
    while n != 1:
        if n % 2 == 0:
            n /= 2
        else:
            n = 3 * n + 1
        yield n


for x in collatz(5):
    print(x)

16
8.0
4.0
2.0
1.0


迭代器的版本如下：

In [77]:
class Collatz(object):
    def __init__(self, start):
        self.value = start

    def __iter__(self):
        return self

    def next(self):
        if self.value == 1:
            raise StopIteration
        elif self.value % 2 == 0:
            self.value = self.value / 2
        else:
            self.value = 3 * self.value + 1
        return self.value
    
for x in collatz(5):
    print(x)

16
8.0
4.0
2.0
1.0


事实上，生成器也是一种迭代器：

In [78]:
x = collatz(5)
x

<generator object collatz at 0x7f93ce863190>

它支持 next 方法，返回下一个 yield 的值：


In [79]:
next(x)

16

In [80]:
next(x)

8.0

`__iter__` 方法返回的是它本身：

In [82]:
x.__iter__()

<generator object collatz at 0x7f93ce863190>

本节完。